In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import warnings
import duckdb
from bs4 import BeautifulSoup
import requests
import unicodedata
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

year = 2025
now = str(datetime.now().date())
print(f"Today's date:", now)

Today's date: 2025-11-19


In [2]:
def email(error, game_id):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA gamelog_rtrvr error"
    body = f"game_id: {game_id}\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [3]:
def remove_accents(text):
    if not isinstance(text, str):
        return text
    # Normalize to NFKD (decomposes characters)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignore non-ASCII characters, then decode back to str
    return text.encode('ascii', 'ignore').decode('ascii')

In [4]:
def retrieve_boxscore(con, soup, gm_type, date, home_team, away_team):
    
    target_ids = [f"box-{home_team}-{gm_type}-basic", f"box-{away_team}-{gm_type}-basic"]
    target_tables = [soup.find("table", {"id": tid}) for tid in target_ids]
    dfs = [pd.read_html(str(tbl), header=1)[0] for tbl in target_tables]
    
    home = dfs[0]
    home['Team'] = home_team
    home['Team_type'] = 'Home'
    home['Opp'] = away_team
    home['Opp_type'] = 'Away'

    away = dfs[1]
    away['Team'] = away_team
    away['Team_type'] = 'Away'
    away['Opp'] = home_team
    away['Opp_type'] = 'Home'
    
    df = pd.concat([home, away])
    df = df[~(df.Starters.isin(['Team Totals', 'Reserves']))].fillna(0).replace("Did Not Play", "DNP")\
            .replace("Did Not Dress", "DNP").replace("Not With Team", "DNP").replace("Player Suspended", "DNP").rename(columns={"Starters": "Player", "3P": "3PM"})
    df["Player"] = df["Player"].apply(lambda x: x.encode("latin1").decode("utf-8"))
    df['Player'] = df['Player'].apply(remove_accents)
    df['Date'] = pd.to_datetime(date, format="%Y%m%d")
    df['Active'] = np.where(df['MP'] == 'DNP', 0, 1)

    df['MP'] = np.where(df.MP == 'DNP', '0:0', df.MP)
    df['mins'] = df.MP.str.split(":").str[0]
    df['secs'] = df.MP.str.split(":").str[1]
    df = df.fillna({'mins': 0, 'secs': 0})
    df['MP'] = df.mins.astype(int) + round(df.secs.astype(int) / 60, 2)
    
    for col in df.columns.difference(['Date', 'Player', 'Team', 'Team_type', 'Opp', 'Opp_type']):
        df[col] = np.where(df[col] == 'DNP', 0, df[col])
        
    for col in ['MP', 'FG%', '3P%', 'FT%', 'GmSc']:
        df[col] = df[col].astype(float)

    for col in df.columns.difference(['Date', 'Player', 'MP', 'Team', 'Team_type', 'Opp', 'Opp_type', 
                                      'FG%', '3P%', 'FT%', 'GmSc', '+/-']):
        df[col] = df[col].astype(int)

    df_pos = pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv")
    df = con.execute("""SELECT * FROM df LEFT JOIN df_pos ON df.Player = df_pos.Player AND df.Team = df_pos.Team""").fetchdf()
        
    df['game_id'] = f"{date}_{away_team}_{home_team}"
    df = df[['game_id', 'Date', 'Team', 'Team_type', 'Opp', 'Opp_type', 'Player', 'Pos', 'Active', 'MP', 
             'FG', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FT', 'FTA',
             'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
             'GmSc', '+/-']]
    
    return df

In [5]:
def fantasy_scoring(df):
    
    # Columns we care about
    stat_cols = ['PTS', 'TRB', 'AST', 'STL', 'BLK']

    # Count how many stats are >= 10 for each player
    df['double_count'] = (df[stat_cols] >= 10).sum(axis=1)

    # Determine double-double or triple-double
    df['DD'] = (df['double_count'] >= 2).astype(int)
    df['TD'] = (df['double_count'] >= 3).astype(int)
    
    df['40Pts'] = (df['PTS'] >= 40).astype(int)
    df['50Pts'] = (df['PTS'] >= 40).astype(int)
    
    df['Fpts'] = (df.PTS * 0.5) + (df.AST * 1) + (df.TRB * 1) + (df.STL * 2) + (df.BLK * 2) + (df.TOV * -1) + (df['3PM'] * 0.5)\
                   + (df.DD * 1) + (df.TD * 2) + (df['40Pts'] * 2) + (df['50Pts'] * 2)
    
    df = df.drop(columns=['double_count', '40Pts', '50Pts'])

    return df

In [6]:
def main():
    con = duckdb.connect(database=":memory:")

    df = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
    df['Date'] = pd.to_datetime(df.Date).astype(str)
    filtrd_df = df[(df.rtrvd == 0) & (df.Date < now)].reset_index(drop=True)
    display(filtrd_df)
    
    for row in range(filtrd_df.shape[0]):
        game = filtrd_df.loc[row]
        date = game.loc['Date'].replace("-", "")
        home_team = game.loc['HomeABV']
        away_team = game.loc['AwayABV']
        url = f"https://www.basketball-reference.com/boxscores/{date}0{home_team}.html"
        print(url)
        
        time.sleep(6)
        html = requests.get(url).text
        soup = BeautifulSoup(html, "lxml")
        
        for gm_type in ['game', 'q1', 'q2', 'q3', 'q4', 'h1', 'h2']:
            if gm_type == 'game':
                file_name = 'season_gamelogs.csv'
            else:
                file_name = f'{gm_type}_season_gamelogs.csv'
                
            try:
                print(f'{gm_type.capitalize()}_Log of {date}_{away_team}_{home_team}')
                df_gm = retrieve_boxscore(con, soup, gm_type, date, home_team, away_team)
                df_gm = fantasy_scoring(df_gm)
                
                try:
                    df_cnct = pd.read_csv(f"../tables/{year}/{file_name}")
                except:
                    df_cnct = pd.DataFrame()
                    
                df_cnct = pd.concat([df_cnct, df_gm])
                df_cnct['Date'] = pd.to_datetime(df_cnct['Date'], format='mixed', errors='coerce')
                df_cnct['Date'] = df_cnct['Date'].dt.strftime("%m/%d/%Y")
                df_cnct.to_csv(f"../tables/{year}/{file_name}", index=False)
                
                if gm_type == 'game':
                    display(df_gm)

            except Exception as e:
                game_id = f"{date}_{away_team}_{home_team}"
                email(e, game_id)
                raise Exception(e)
        
        df_save_indx = df[(df.Date == game.loc['Date']) & (df.AwayABV == away_team) & (df.HomeABV == home_team)].index
        df.loc[df_save_indx, 'rtrvd'] = 1
        df.to_csv(f"../tables/{year}/nba_schedule.csv", index=False)

    con.close()
    
main()

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
0,2025-11-18,7:00p,Golden State Warriors,GSW,Orlando Magic,ORL,Kia Center,0,0,0
1,2025-11-18,7:30p,Detroit Pistons,DET,Atlanta Hawks,ATL,State Farm Arena,1,0,0
2,2025-11-18,7:30p,Boston Celtics,BOS,Brooklyn Nets,BRK,Barclays Center,0,0,0
3,2025-11-18,8:00p,Memphis Grizzlies,MEM,San Antonio Spurs,SAS,Frost Bank Center,0,0,0
4,2025-11-18,10:30p,Utah Jazz,UTA,Los Angeles Lakers,LAL,Crypto.com Arena,0,0,0
5,2025-11-18,11:00p,Phoenix Suns,PHO,Portland Trail Blazers,POR,Moda Center,0,0,0


https://www.basketball-reference.com/boxscores/202511180ORL.html
Game_Log of 20251118_GSW_ORL


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Brandin Podziemski,SG,1,26.52,2,7,0.286,1,2,0.500,0,0,0.000,1,5,6,0,0,0,0,2,5,2.3,-7,0,0,9.0
1,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Buddy Hield,SG,1,15.20,1,3,0.333,0,1,0.000,0,0,0.000,0,0,0,0,0,0,1,0,2,-0.7,-5,0,0,0.0
2,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Quinten Post,PF,1,12.15,1,4,0.250,0,3,0.000,0,0,0.000,2,1,3,1,0,0,0,0,2,2.0,0,0,0,5.0
3,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Jimmy Butler,SF,1,38.23,10,16,0.625,0,1,0.000,13,15,0.867,1,6,7,4,3,0,3,0,33,30.3,-1,0,0,30.5
4,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Jonathan Kuminga,PF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
5,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Draymond Green,PF,1,31.40,5,8,0.625,2,5,0.400,0,0,0.000,0,6,6,6,0,2,3,4,12,11.2,+9,0,0,20.0
6,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Gary Payton II,SG,1,10.25,2,3,0.667,0,0,0.000,0,0,0.000,1,0,1,1,0,0,0,2,4,3.3,-15,0,0,4.0
7,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Moses Moody,SG,1,29.57,2,6,0.333,1,4,0.250,1,2,0.500,1,3,4,2,2,0,3,3,6,3.0,-3,0,0,10.5
8,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Will Richard,SG,1,15.30,2,2,1.000,1,1,1.000,1,2,0.500,0,0,0,2,1,0,1,3,6,5.2,-4,0,0,6.5
9,20251118_GSW_ORL,2025-11-18,GSW,Away,ORL,Home,Stephen Curry,PG,1,34.22,12,23,0.522,7,15,0.467,3,5,0.600,0,3,3,9,3,0,5,2,34,26.3,-3,0,0,33.5


Q1_Log of 20251118_GSW_ORL
Q2_Log of 20251118_GSW_ORL
Q3_Log of 20251118_GSW_ORL
Q4_Log of 20251118_GSW_ORL
H1_Log of 20251118_GSW_ORL
H2_Log of 20251118_GSW_ORL
https://www.basketball-reference.com/boxscores/202511180ATL.html
Game_Log of 20251118_DET_ATL


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Onyeka Okongwu,C,1,33.58,8,13,0.615,4,7,0.571,1,1,1.000,0,1,1,3,1,2,5,6,21,12.5,-22,0,0,17.5
1,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Dyson Daniels,SG,1,37.58,6,9,0.667,0,0,0.000,0,0,0.000,4,5,9,6,3,1,3,1,12,16.9,+4,0,0,26.0
2,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Mouhamed Gueye,PF,1,21.40,5,10,0.500,1,5,0.200,0,0,0.000,7,4,11,1,0,1,1,3,11,11.3,+17,1,0,20.0
3,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Jalen Johnson,SF,1,39.10,8,18,0.444,3,8,0.375,6,6,1.000,0,8,8,9,3,1,4,2,25,23.2,-7,0,0,35.0
4,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Nickeil Alexander-Walker,SG,1,36.72,9,20,0.450,1,9,0.111,5,6,0.833,0,3,3,3,0,0,0,4,24,14.6,-19,0,0,18.5
5,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Zaccharie Risacher,SF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
6,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Luke Kennard,SG,1,17.08,1,2,0.500,1,2,0.500,0,0,0.000,0,0,0,5,1,0,1,3,3,4.3,+4,0,0,8.0
7,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Vit Krejci,PG,1,36.10,3,9,0.333,3,8,0.375,0,0,0.000,0,3,3,2,0,0,2,3,9,3.0,+1,0,0,9.0
8,20251118_DET_ATL,2025-11-18,ATL,Home,DET,Away,Asa Newell,PF,1,1.92,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,-4,0,0,0.0
9,20251118_DET_ATL,2025-11-18,DET,Away,ATL,Home,Duncan Robinson,SF,1,30.68,5,10,0.500,4,9,0.444,0,0,0.000,1,2,3,3,1,0,2,1,14,11.0,+4,0,0,15.0


...

Q1_Log of 20251118_MEM_SAS
Q2_Log of 20251118_MEM_SAS
Q3_Log of 20251118_MEM_SAS
Q4_Log of 20251118_MEM_SAS
H1_Log of 20251118_MEM_SAS
H2_Log of 20251118_MEM_SAS
https://www.basketball-reference.com/boxscores/202511180LAL.html
Game_Log of 20251118_UTA_LAL


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Rui Hachimura,PF,1,25.70,3,8,0.375,0,2,0.000,0,0,0.000,2,2,4,1,0,0,0,1,6,3.9,+14,0,0,8.0
1,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Jake LaRavia,PF,1,25.50,6,10,0.600,2,5,0.400,2,3,0.667,0,4,4,2,1,0,0,1,16,14.2,+12,0,0,17.0
2,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Jarred Vanderbilt,PF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
3,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Deandre Ayton,C,1,30.00,10,13,0.769,0,0,0.000,0,2,0.000,4,10,14,1,1,1,1,2,20,20.5,+10,1,0,29.0
4,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Dalton Knecht,SF,1,3.55,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,-7,0,0,0.0
5,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Marcus Smart,SG,1,17.15,2,5,0.400,1,4,0.250,0,0,0.000,1,2,3,1,0,0,0,3,5,3.1,+11,0,0,7.0
6,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Austin Reaves,SG,1,33.43,7,11,0.636,1,4,0.250,11,12,0.917,1,4,5,1,1,0,4,3,26,19.1,+10,0,0,17.5
7,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Jaxson Hayes,C,1,15.03,3,3,1.000,0,0,0.000,2,3,0.667,2,1,3,0,1,0,0,1,8,9.0,+13,0,0,9.0
8,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Luka Doncic,PG,1,34.23,11,22,0.500,2,10,0.200,13,16,0.813,0,5,5,10,4,0,8,4,37,27.7,+10,1,0,35.5
9,20251118_UTA_LAL,2025-11-18,LAL,Home,UTA,Away,Bronny James,SG,1,3.55,1,1,1.000,1,1,1.000,0,0,0.000,0,0,0,2,0,0,3,1,3,0.7,-7,0,0,1.0


Q1_Log of 20251118_UTA_LAL
Q2_Log of 20251118_UTA_LAL
Q3_Log of 20251118_UTA_LAL
Q4_Log of 20251118_UTA_LAL
H1_Log of 20251118_UTA_LAL
H2_Log of 20251118_UTA_LAL
https://www.basketball-reference.com/boxscores/202511180POR.html
Game_Log of 20251118_PHO_POR


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Devin Booker,SG,1,29.05,6,13,0.462,1,2,0.500,6,6,1.000,1,5,6,5,3,0,1,2,19,19.2,+17,0,0,26.0
1,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Royce O'Neale,SF,1,31.17,3,10,0.300,3,10,0.300,0,0,0.000,0,5,5,1,2,0,1,2,9,5.6,+19,0,0,15.0
2,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Collin Gillespie,PG,1,28.18,6,11,0.545,4,9,0.444,3,3,1.000,0,3,3,6,1,0,3,3,19,15.6,+1,0,0,19.5
3,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Ryan Dunn,SF,1,26.53,4,8,0.500,1,1,1.000,0,0,0.000,0,3,3,4,5,0,1,4,9,11.1,+23,0,0,21.0
4,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Oso Ighodaro,PF,1,16.08,6,7,0.857,0,0,0.000,2,2,1.000,3,1,4,1,0,0,1,2,14,12.8,+7,0,0,11.0
5,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Nick Richards,C,1,10.65,4,7,0.571,0,0,0.000,0,0,0.000,2,0,2,0,0,1,0,2,8,6.0,-5,0,0,8.0
6,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Mark Williams,C,1,21.27,7,9,0.778,0,0,0.000,1,2,0.500,4,2,6,1,2,1,2,3,15,14.7,+15,0,0,18.5
7,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Nigel Hayes-Davis,SF,1,3.60,0,3,0.000,0,1,0.000,0,0,0.000,2,1,3,0,0,0,0,1,0,-0.8,-3,0,0,3.0
8,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Jordan Goodwin,PG,1,21.80,4,10,0.400,2,5,0.400,0,0,0.000,2,2,4,1,3,1,1,2,10,9.2,+1,0,0,18.0
9,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Dillon Brooks,SF,1,25.63,5,11,0.455,1,6,0.167,1,2,0.500,1,1,2,4,3,0,3,3,12,8.5,+16,0,0,15.5


Q1_Log of 20251118_PHO_POR
Q2_Log of 20251118_PHO_POR
Q3_Log of 20251118_PHO_POR
Q4_Log of 20251118_PHO_POR
H1_Log of 20251118_PHO_POR
H2_Log of 20251118_PHO_POR
